# Check if everything is in place

## Weaviate Python Client v4
> This notebook was created with Weaviate `1.26` and the Weaviate Client `4.7`

Run the below command to check if you run the latest version of the Weaviate Python Client v4.

In [1]:
!pip show weaviate-client

Name: weaviate-client
Version: 4.16.9
Summary: A python native Weaviate client
Home-page: https://github.com/weaviate/weaviate-python-client
Author: Weaviate
Author-email: hello@weaviate.io,
License: BSD 3-clause
Location: /Users/scottaskinosie/Documents/weaviate/workshops/nato/weaviate-workshop/.venv/lib/python3.11/site-packages
Requires: authlib, deprecation, grpcio, httpx, protobuf, pydantic, validators
Required-by: distyll-info


## Get keys and urls

> You can update your env variables in the `.env` file at the root of the project.

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")

print(WEAVIATE_URL[:10])
print(WEAVIATE_KEY[:10])

cwzujlbgtk
L1FUbzRPaU


## Connect to Weaviate

In [3]:
import weaviate
from weaviate.classes.init import Auth
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=2, query=45, insert=120),  # Values in seconds
    # )
)

client.is_ready()

True

## Display the available modules

> You should be able to see 'generative-openai' and 'text2vec-openai', plus many other modules.

In [4]:
client.get_meta()

{'grpcMaxMessageSize': 104858000,
 'hostname': 'http://[::]:8080',
 'modules': {'backup-gcs': {'bucketName': 'weaviate-wcs-prod-cust-us-west3-workloads-backups',
   'rootName': '7166548e-5046-4e4f-b3c5-097696f1ac18'},
  'generative-anthropic': {'documentationHref': 'https://docs.anthropic.com/en/api/getting-started',
   'name': 'Generative Search - Anthropic'},
  'generative-anyscale': {'documentationHref': 'https://docs.anyscale.com/endpoints/overview',
   'name': 'Generative Search - Anyscale'},
  'generative-aws': {'documentationHref': 'https://docs.aws.amazon.com/bedrock/latest/APIReference/welcome.html',
   'name': 'Generative Search - AWS'},
  'generative-cohere': {'documentationHref': 'https://docs.cohere.com/reference/chat',
   'name': 'Generative Search - Cohere'},
  'generative-databricks': {'documentationHref': 'https://docs.databricks.com/en/machine-learning/foundation-models/api-reference.html#completion-task',
   'name': 'Generative Search - Databricks'},
  'generative-fr

### Close the client
When you are done with the client, you should close it to release the resources.

In [5]:
client.close()